In [3]:
import json
import requests


patch_info_edited = json.load(open('patch_info_edited.json'))
embeddings = json.load(open('embeddings.json'))



In [32]:
len(patch_info_edited), len(embeddings)

(27200, 27200)

In [34]:
index = -1

for i, patch in enumerate(patch_info_edited):
    if patch == '3557_D23-00412-29-30_ .tiff_5120_5120_256_256_2.png':
        index = i
        print(index)
        break

if index == -1:
    print('not found')
    exit()

find_embedding = embeddings[index]

20885


In [6]:
find_embedding[:10]

[0.1342412680387497,
 -1.726455569267273,
 1.4469287395477295,
 -2.0141983032226562,
 -0.8886911869049072,
 0.07538362592458725,
 -1.3197290897369385,
 1.4392619132995605,
 -2.3907861709594727,
 -1.8447942733764648]

In [8]:
import sys
sys.path.append('..')
from MDI_RAG_Image2Image_Research.src.utils.encoder import WSI_Image_UNI_Encoder

In [13]:
from PIL import Image
from io import BytesIO


def request_image(query_img_path):
    if "http" in query_img_path:
        response = requests.get(query_img_path, verify=False)
        query_image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        query_image = Image.open(query_img_path).convert("RGB")
    return query_image

In [14]:
query_img_path = ''

In [15]:
query_image = request_image(query_img_path)
image_encoder = WSI_Image_UNI_Encoder()

query_embedding = image_encoder.encode_image(query_image)

/hpc2hdd/home/ysi538/miniconda3/envs/yhre/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.108.5.177'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


cuda


In [16]:
query_embedding[:10]

[-0.47307372093200684,
 -0.16042059659957886,
 0.391374796628952,
 -0.6895554065704346,
 -0.002383788116276264,
 1.1413025856018066,
 0.1865881085395813,
 1.336168885231018,
 2.190131425857544,
 0.2908326983451843]

In [ ]:
import openslide
import os

def get_slide_info(wsi_name):
    
    filepath = os.path.join('caches/wsi_image/', wsi_name)
    filepath = os.path.join('/hpc2hdd/home/ysi538/retrieval/caches/wsi_image/', wsi_name)
    if not os.path.isfile(filepath):
        msg = {"error": "No such file"}
        print(msg)
        return msg
    # metadata['location'] = filepath
    # print(f"Loading {filepath}")
    if os.path.getsize(filepath) < 5 * 1024 * 1024:
        msg = {"error": "File size less than 5MB"}
        # print(msg)
        return None,None
    try:
        slide = openslide.OpenSlide(filepath)
        slide_properties = slide.properties
    except BaseException as error:
        msg = {"type": "Openslide", "error": str(error)}
        # print(msg)
        return None,None
        
    return slide, slide_properties

In [ ]:
wsi_name = ''
slide, slide_info = get_slide_info(wsi_name)

TIFFReadDirectory: Warning, Unknown field with tag 65500 (0xffdc) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 65500 (0xffdc) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 65500 (0xffdc) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 65500 (0xffdc) encountered.


In [23]:
level = 2
x = 5120
y = 5120
width = 256
height = 256
ratio = slide.level_downsamples[level]
x = int(x * ratio)
y = int(y * ratio)


region = slide.read_region((x, y), level, (width, height))
region = region.convert("RGB")
# region.show()
region_embedding = image_encoder.encode_image(region)

In [25]:
region_embedding[:]

[-0.47307372093200684,
 -0.16042059659957886,
 0.391374796628952,
 -0.6895554065704346,
 -0.002383788116276264,
 1.1413025856018066,
 0.1865881085395813,
 1.336168885231018,
 2.190131425857544,
 0.2908326983451843,
 -1.620772361755371,
 -1.448141098022461,
 2.2439963817596436,
 0.3827891945838928,
 1.804341435432434,
 -1.561109185218811,
 2.7370784282684326,
 0.43201950192451477,
 0.6125331521034241,
 -0.02567044086754322,
 -0.9585774540901184,
 -1.6406387090682983,
 0.367338627576828,
 0.3909333050251007,
 -1.418588399887085,
 -0.2847904860973358,
 0.8118169903755188,
 1.1251264810562134,
 -0.46629267930984497,
 -0.24440248310565948,
 0.35422268509864807,
 0.1787177175283432,
 1.0966899394989014,
 0.0351790226995945,
 -0.8687563538551331,
 -1.6246205568313599,
 1.0462745428085327,
 -2.274479627609253,
 -0.12460951507091522,
 -0.36347851157188416,
 -2.2247297763824463,
 3.6531436443328857,
 -0.8353205919265747,
 0.7324941158294678,
 -2.088430166244507,
 1.282546043395996,
 -3.376128673

In [28]:
a = [1,2,3]
b = [1,2,3]
a == b

True

In [31]:
type(find_embedding[0])

float

In [29]:


for i, embedding in enumerate(embeddings):
    if embedding == region_embedding:
        print(i)
        break


In [ ]:
from MDI_RAG_Image2Image_Research.src.build.basic.Untitled import WSIUNIEncoder



In [ ]:
emb2[:10]